# Opdracht 2

In [58]:
import random
import simpy

N_helpers = [1, 2, 4]  # Number of machines in the queue
waitTime = 10  # Minutes it takes to help a customer
lambdaIAT = 5  # Create a customer every ~7 minutes
SIM_TIME = 60  # Simulation time in minutes


class Queue(object):
    def __init__(self, env, N, waitTime):
        self.env = env
        self.machine = simpy.Resource(env, N)
        self.waitTime = waitTime
        self.customerHelped = 0
        self.helperN = N

    def helped(self, customer, customerWait=waitTime):
        yield self.env.timeout(customerWait)


#         print(f"{customer} helped at {self.env.now}")


def customer(env, name, cw):
    customerWait = random.randint(waitTime - 2, waitTime + 2)
    #     print('%s arrives at the queue at %.2f.' % (name, env.now))
    enterQueue = env.now
    with cw.machine.request() as request:
        yield request
        
        #         print('%s turn begins at %.2f.' % (name, env.now))
        customerStat[cw.helperN].append(env.now - enterQueue)
        #         yield env.process(cw.helped(name))
        yield env.process(cw.helped(name, customerWait=customerWait))

#         print('%s leaves the queue at %.2f.' % (name, env.now))


def setup(env, N, waitTime, lambdaIAT):
    queue = Queue(env, N, waitTime)

    # Create more customers while the simulation is running
    while queue.customerHelped < 10:
        yield env.timeout(random.randint(lambdaIAT - 2, lambdaIAT + 2))
        queue.customerHelped += 1
        env.process(customer(env, 'customer %d' % queue.customerHelped, queue))


customerStat = {}
for j in range(1000):
    for N in N_helpers:
        #     print(f"******** N={N} Helpers ********")
        customerStat.setdefault(N, [])
        env = simpy.Environment()
        env.process(setup(env, N, waitTime, lambdaIAT))
        env.run()

[
    print(f"run{N} average wait: {np.mean(customerStat[N])}")
    for N in customerStat
]

run1 average wait: 22.5684
run2 average wait: 1.5129
run4 average wait: 0.0


[None, None, None]